### Librería y conexión

In [1]:
import sqlalchemy

# engine = sqlalchemy.create_engine('sqlite:///data.db', echo=True) # echo=True prints out the SQL commands
engine = sqlalchemy.create_engine('sqlite:///:memory:', echo=True) # memory is for testing

# Declarative Base
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Data types
from sqlalchemy import Column, Integer, String


### Definición de modelos

In [2]:
# Model definition
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

### Creación de tablas a partir de modelos

In [3]:
# Table configuration
Base.metadata.create_all(engine)

2022-12-13 15:17:14,834 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-13 15:17:14,838 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-12-13 15:17:14,840 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-13 15:17:14,843 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2022-12-13 15:17:14,845 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-13 15:17:14,851 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2022-12-13 15:17:14,853 INFO sqlalchemy.engine.Engine [no key 0.00184s] ()
2022-12-13 15:17:14,855 INFO sqlalchemy.engine.Engine COMMIT


### Creación de sesión

In [4]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

### Prueba de modelos

In [5]:
# Create a new user
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
session.add(ed_user)
ed_user

<User(name='ed', fullname='Ed Jones', password='edspassword')>

In [6]:
user = session.query(User).filter_by(name='ed').first()
if user:
    print(user)
else:
    print('No user found')

2022-12-13 15:17:15,102 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-13 15:17:15,110 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2022-12-13 15:17:15,113 INFO sqlalchemy.engine.Engine [generated in 0.00327s] ('ed', 'Ed Jones', 'edspassword')
2022-12-13 15:17:15,125 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2022-12-13 15:17:15,127 INFO sqlalchemy.engine.Engine [generated in 0.00215s] ('ed', 1, 0)
<User(name='ed', fullname='Ed Jones', password='edspassword')>


In [7]:
# Multiple modifications in a single commit
session.add_all([
    User(name='wendy', fullname='Wendy Williams', password='foobar'),
    User(name='mary', fullname='Mary Contrary', password='xxg527'),
    User(name='fred', fullname='Fred Flinstone', password='blah')])

ed_user.password = 'f8s7ccs'
ed_user.fullname = 'Eduardo Jones'

In [8]:
session.commit()

2022-12-13 15:17:15,283 INFO sqlalchemy.engine.Engine UPDATE users SET fullname=?, password=? WHERE users.id = ?
2022-12-13 15:17:15,285 INFO sqlalchemy.engine.Engine [generated in 0.00237s] ('Eduardo Jones', 'f8s7ccs', 1)
2022-12-13 15:17:15,288 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2022-12-13 15:17:15,290 INFO sqlalchemy.engine.Engine [cached since 0.1799s ago] ('wendy', 'Wendy Williams', 'foobar')
2022-12-13 15:17:15,292 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2022-12-13 15:17:15,294 INFO sqlalchemy.engine.Engine [cached since 0.1836s ago] ('mary', 'Mary Contrary', 'xxg527')
2022-12-13 15:17:15,296 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2022-12-13 15:17:15,300 INFO sqlalchemy.engine.Engine [cached since 0.1896s ago] ('fred', 'Fred Flinstone', 'blah')
2022-12-13 15:17:15,305 INFO sqlalchemy.engine.Engine COMMIT


### Retroceder cambios


In [9]:
# Cambios nuevos
ed_user.fullname = 'Ed Jones'

fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)

session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()


2022-12-13 15:17:15,389 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-13 15:17:15,395 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.password AS users_password 
FROM users 
WHERE users.id = ?
2022-12-13 15:17:15,399 INFO sqlalchemy.engine.Engine [generated in 0.00486s] (1,)
2022-12-13 15:17:15,404 INFO sqlalchemy.engine.Engine UPDATE users SET fullname=? WHERE users.id = ?
2022-12-13 15:17:15,406 INFO sqlalchemy.engine.Engine [generated in 0.00223s] ('Ed Jones', 1)
2022-12-13 15:17:15,408 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2022-12-13 15:17:15,410 INFO sqlalchemy.engine.Engine [cached since 0.3s ago] ('fakeuser', 'Invalid', '12345')
2022-12-13 15:17:15,415 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2022-12-13 15:17:15,418 INFO sqlal

[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>,
 <User(name='fakeuser', fullname='Invalid', password='12345')>]

In [10]:
# Rollback
session.rollback()
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()


2022-12-13 15:17:15,495 INFO sqlalchemy.engine.Engine ROLLBACK
2022-12-13 15:17:15,502 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-13 15:17:15,508 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2022-12-13 15:17:15,510 INFO sqlalchemy.engine.Engine [cached since 0.09544s ago] ('ed', 'fakeuser')


[<User(name='ed', fullname='Eduardo Jones', password='f8s7ccs')>]

### Consultas

In [11]:
for instance in session.query(User).order_by(User.id):
    print(instance)

2022-12-13 15:17:15,632 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
2022-12-13 15:17:15,635 INFO sqlalchemy.engine.Engine [generated in 0.00407s] ()
<User(name='ed', fullname='Eduardo Jones', password='f8s7ccs')>
<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>
<User(name='fred', fullname='Fred Flinstone', password='blah')>


In [12]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2022-12-13 15:17:15,737 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2022-12-13 15:17:15,741 INFO sqlalchemy.engine.Engine [generated in 0.00405s] ()
ed Eduardo Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


### Filtro

In [13]:
# equals
for user in session.query(User).filter(User.name=='ed'):
    print(user)

2022-12-13 15:17:15,849 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
2022-12-13 15:17:15,851 INFO sqlalchemy.engine.Engine [generated in 0.00294s] ('ed',)
<User(name='ed', fullname='Eduardo Jones', password='f8s7ccs')>


In [14]:
# not equals
for user in session.query(User).filter(User.name!='ed'):
    print(user)

2022-12-13 15:17:15,951 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name != ?
2022-12-13 15:17:15,954 INFO sqlalchemy.engine.Engine [generated in 0.00342s] ('ed',)
<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>
<User(name='fred', fullname='Fred Flinstone', password='blah')>


In [15]:
# like
for user in session.query(User).filter(User.name.like('%ed%')):
    print(user)

2022-12-13 15:17:16,033 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ?
2022-12-13 15:17:16,037 INFO sqlalchemy.engine.Engine [generated in 0.00364s] ('%ed%',)
<User(name='ed', fullname='Eduardo Jones', password='f8s7ccs')>
<User(name='fred', fullname='Fred Flinstone', password='blah')>


In [16]:
# in
for user in session.query(User).filter(User.name.in_(['ed', 'wendy', 'gil'])):
    print(user)
    

2022-12-13 15:17:16,142 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?, ?)
2022-12-13 15:17:16,146 INFO sqlalchemy.engine.Engine [cached since 0.732s ago] ('ed', 'wendy', 'gil')
<User(name='ed', fullname='Eduardo Jones', password='f8s7ccs')>
<User(name='wendy', fullname='Wendy Williams', password='foobar')>


In [17]:
# not in
for user in session.query(User).filter(~User.name.in_(['ed', 'wendy', 'gil'])):
    print(user)

2022-12-13 15:17:16,261 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE (users.name NOT IN (?, ?, ?))
2022-12-13 15:17:16,265 INFO sqlalchemy.engine.Engine [generated in 0.00407s] ('ed', 'wendy', 'gil')
<User(name='mary', fullname='Mary Contrary', password='xxg527')>
<User(name='fred', fullname='Fred Flinstone', password='blah')>


In [18]:
# And
for user in session.query(User).filter(User.name=='ed', User.fullname=='Eduardo Jones'):
    print(user)

2022-12-13 15:17:16,424 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2022-12-13 15:17:16,429 INFO sqlalchemy.engine.Engine [generated in 0.00451s] ('ed', 'Eduardo Jones')
<User(name='ed', fullname='Eduardo Jones', password='f8s7ccs')>


In [19]:
# Or
for user in session.query(User).filter(sqlalchemy.or_(User.name=='ed', User.name=='wendy')):
    print(user)

2022-12-13 15:17:16,653 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? OR users.name = ?
2022-12-13 15:17:16,658 INFO sqlalchemy.engine.Engine [generated in 0.00509s] ('ed', 'wendy')
<User(name='ed', fullname='Eduardo Jones', password='f8s7ccs')>
<User(name='wendy', fullname='Wendy Williams', password='foobar')>


### Relaciones

In [20]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

In [21]:
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")

Base.metadata.create_all(engine)

2022-12-13 15:17:17,006 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-13 15:17:17,009 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-12-13 15:17:17,012 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-13 15:17:17,017 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("addresses")
2022-12-13 15:17:17,019 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-13 15:17:17,023 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("addresses")
2022-12-13 15:17:17,024 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-13 15:17:17,028 INFO sqlalchemy.engine.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2022-12-13 15:17:17,030 INFO sqlalchemy.engine.Engine [no key 0.00270s] ()
2022-12-13 15:17:17,034 INFO sqlalchemy.engine.Engine COMMIT


In [22]:
ed_user.addresses = [Address(email_address="hola@col.com"), Address(email_address="rock@lml.com")]

session.add(ed_user)
session.commit()

2022-12-13 15:17:17,185 INFO sqlalchemy.engine.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id ORDER BY addresses.id
2022-12-13 15:17:17,188 INFO sqlalchemy.engine.Engine [generated in 0.00322s] (1,)
2022-12-13 15:17:17,196 INFO sqlalchemy.engine.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2022-12-13 15:17:17,199 INFO sqlalchemy.engine.Engine [generated in 0.00405s] ('hola@col.com', 1)
2022-12-13 15:17:17,203 INFO sqlalchemy.engine.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2022-12-13 15:17:17,205 INFO sqlalchemy.engine.Engine [cached since 0.009338s ago] ('rock@lml.com', 1)
2022-12-13 15:17:17,208 INFO sqlalchemy.engine.Engine COMMIT


In [23]:
# print all users and their addresses
for u, a in session.query(User, Address).filter(User.id==Address.user_id):
    print(u)
    print(a)

2022-12-13 15:21:19,320 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-13 15:21:19,327 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users, addresses 
WHERE users.id = addresses.user_id
2022-12-13 15:21:19,330 INFO sqlalchemy.engine.Engine [generated in 0.00257s] ()
<User(name='ed', fullname='Eduardo Jones', password='f8s7ccs')>
<Address(email_address='hola@col.com')>
<User(name='ed', fullname='Eduardo Jones', password='f8s7ccs')>
<Address(email_address='rock@lml.com')>
